In [1]:
import os
import torch
import torchaudio
import sys

sys.path.append('./src_torch')

import soundfile as sf

from src_torch.separation import FastMNMF2
from src_torch.Base import MultiSTFT


In [4]:
n_source = 3
n_basis = 32
device = "cuda:1" if torch.cuda.is_available() else "cpu"
init_SCM = "twostep"
n_bit = 32
algo = "IP"
n_iter_init = 60
g_eps = 5e-3

input_fname = "./mic_3.wav"
n_mic = 4
n_fft = 2048
n_iter = 500


separater = FastMNMF2(
    n_source=n_source,
    n_basis=n_basis,
    device=device,
    init_SCM=init_SCM,
    n_bit=n_bit,
    algo=algo,
    n_iter_init=n_iter_init,
    g_eps=g_eps,
)

wav, sample_rate = torchaudio.load(input_fname, channels_first=False)
wav /= torch.abs(wav).max() * 1.2
M = min(len(wav), n_mic)
spec_FTM = MultiSTFT(wav[:, :M], n_fft=n_fft)

separater.file_id = input_fname.split("/")[-1].split(".")[0]
separater.load_spectrogram(spec_FTM, sample_rate)
separater.solve(
    n_iter=n_iter,
    save_dir="./",
    save_likelihood=False,
    save_param=False,
    save_wav=True,
    interval_save=5,
)

Update FastMNMF2_IP-M=1-S=3-F=1025-K=2-init=circular-g=0.005-bit=32-intv_norm=10  60 times ...


100%|██████████| 60/60 [00:01<00:00, 55.63it/s]


Update FastMNMF2_IP-M=1-S=3-F=1025-K=32-init=twostep_60it-g=0.005-bit=32-intv_norm=10-ID=mic_3  440 times ...


100%|██████████| 440/440 [00:09<00:00, 48.79it/s]


In [4]:
input_fname = "FastMNMF2_IP-sep-M=2-S=3-F=513-K=16-init=twostep_30it-g=0.005-bit=32-intv_norm=10-ID=test_audio-100.wav"
wav, sample_rate = torchaudio.load(input_fname, channels_first=False)
wav = wav.t()
for i in range(n_source):
    sf.write(f"sep_{i}.wav", wav[i].numpy(), sample_rate)
